# Conceptual Search Quickstart

This notebook demonstrates how to use the new **Conceptual Search** functionality in the `intugle` library. The process involves two main steps:

1.  **Generate a Data Product Plan:** You provide a high-level goal in natural language (e.g., "I want a report on patient allergies"), and the Planner Agent generates a structured CSV file (`attributes.csv`) detailing the required dimensions and measures.
2.  **Build the Data Product:** The Builder Agent takes the generated plan and maps each attribute to the most relevant columns in your data sources, creating a final mapping file (`column_logic_results.csv`).

**Prerequisite:** This workflow requires a built `SemanticModel`, as the agents rely on the manifest YAML files it generates.

### Step 1: Build the Semantic Model

First, we need to build a `SemanticModel` from our data sources. This will profile the data, discover relationships, and generate the manifest files that the Conceptual Search agents need to understand your data landscape.

In [ ]:
from dotenv import load_dotenv
load_dotenv(override=True)

from intugle import SemanticModel

# Define the paths to your datasets
# For this example, we'll use the sample healthcare data included in the repo.
datasets = {
    "patients": {"path": "../sample_data/healthcare/patients.csv", "type": "csv"},
    "allergies": {"path": "../sample_data/healthcare/allergies.csv", "type": "csv"},
    "encounters": {"path": "../sample_data/healthcare/encounters.csv", "type": "csv"},
    "conditions": {"path": "../sample_data/healthcare/conditions.csv", "type": "csv"}
}

# Build the semantic model
# This will generate the necessary YAML files in your project root directory.
sm = SemanticModel(datasets, domain="Healthcare")
sm.build()

print("Semantic Model has been successfully built.")

### Step 2: Generate a Data Product Plan

Now that the semantic model is built, we can use the `ConceptualSearch` class to generate a plan for our desired data product. We'll define our goal in a simple, natural language query.

In [ ]:
from intugle import DataProduct

data_product = DataProduct()


query = "patient 360"


data_product_plan = await data_product.plan(query, "patient 360 view")

# Display the plan. This is a special object that can be modified in code.
data_product_plan


In [ ]:
# data_product_plan.disable_attribute("Date of Birth")

### Step 3: Build the Data Product (Map Plan to Columns)

With the plan generated, we can now pass it to the Builder Agent. This agent will intelligently search the semantic model to find the best table and column for each attribute in the plan.

In [ ]:
# import pandas as pd
# from intugle.core.conceptual_search.search import ConceptualSearch
# conceptual_search = ConceptualSearch(force_recreate=False)


# plan_df = pd.read_csv("attributes.csv")

# The 'plan_df' is the DataFrame we generated in the previous step
if data_product_plan is not None:
    etl_model = await data_product.create_etl_model_from_plan(data_product_plan)

print(etl_model.model_dump_json(indent=2))

In [ ]:
result_dataset = data_product.build(etl_model)

print("-- Generated SQL Query --")
print(result_dataset.sql_query)

print("\n-- Resulting Data (First 5 Rows) --")
result_dataset.to_df().head()